# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

When solving this task, we expect you'll face (and successfully deal with) some problems or make up the ideas of the model improvement. Some of them are: 

- solving a problem of n-grams frequencies storing for a large corpus;
- taking into account keyboard layout and associated misspellings;
- efficiency improvement to make the solution faster;
- ...

Please don't forget to describe such cases, and what you decided to do with them, in the Justification section.

##### IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [99]:
# Your code here
import re
from math import exp, log, e

In [100]:
def normalize(text, to_string=False):
    # delete everything except words and numbers
    text = re.findall(r'\w+', text.lower())
    if to_string:
        text = ' '.join(text)
    return text

In [101]:
def create_stats(n=2):
    stats = {}
    word_set = set()
    with open('fivegrams.txt') as f:
        text = f.read().split('\n')
    for words in text:
        words = words.split()
        if len(words) != 6: break
        cnt = int(words.pop(0))
        word_set.update(words)
        words = [0] * n + words + [0] * n
        for i in range(n, len(words) - n):
            bag = words[i - n:i] + words[i + 1:i + n + 1]
            for _ in range(n):
                bg = tuple(bag)
                if bg not in stats:
                    stats[bg] = {0: 0}
                stats[bg][words[i]] = stats[bg].get(words[i], 0) + cnt
                stats[bg][0] += cnt
                bag.pop(0)
                bag.pop()
    return stats, word_set


stats, word_set = create_stats()

In [102]:
def get_statistics(wrds):
    bag = list(wrds)
    candidates = {word: e for word in word_set}
    while bag:
        res = stats.get(tuple(bag))
        if res:
            sum = res[0] + 1
            for word, num in res.items():
                if word == 0: continue
                candidates[word] = candidates[word] / (sum - num) * sum
        bag.pop(0)
        bag.pop()
    return candidates

In [103]:
def dist(word1, word2):
    ln1 = len(word1) + 1
    ln2 = len(word2) + 1
    dst = [[0] * ln2 for _ in range(ln1)]
    for i in range(ln1): dst[i][0] = i
    for i in range(ln2): dst[0][i] = i
    for i in range(1, ln1):
        for j in range(1, ln2):
            dst[i][j] = min(dst[i][j - 1], dst[i - 1][j], dst[i - 1][j - 1])
            if word1[i - 1] == word2[j - 1]:
                dst[i][j] = dst[i - 1][j - 1]
            else:
                dst[i][j] = min(dst[i - 1][j], dst[i][j - 1], dst[i - 1][j - 1]) + 1
    return dst[-1][-1]

In [104]:
def correct_word(wrds):
    i = len(wrds) // 2
    distances = {}
    for word in word_set:
        distances[word] = dist(word, wrds[i])
    wrds.pop(i)
    stat = get_statistics(wrds)
    stat = max((log(stat[word]) * exp(-distances[word]), word) for word in stat)
    return stat[1]

In [105]:
def correct_text(text, n=2):
    text = normalize(text)
    text = [0] * n + text + [0] * n
    for i in range(n, len(text) - n):
        if text[i] not in word_set:
            text[i] = correct_word(text[i - n:i + n + 1])
    return ' '.join(text[n:len(text) - n])

In [106]:
correct_text('tve forest was alive wth the saunds')

'the forest was alive with the sounds'

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

*Your text here...*

## Dataset
The dataset of fivegrams was used for this task.
This dataset contains >1M lines of fivegrams and >25K unique words.
## Algorithm
#### Statistics
For this task were used different levels of n-grams: 3-grams, 5-grams, ..., (n * 2 + 1) - grams(n chosen by user, in this code used n = 2).
Each n-gram consists of a context words on the left and on the right and the target word in the center, contextual words are used as keys and target words with their count as values.

Example:
"a couple of days later"
"a couple" is left context and "days later" is right context
(couple, days): {of: 42} for 3-grams
(a, couple, days, later): {of: 24} for 5-grams

for target word "days" we have not enough words, so we add 0 paddings.
(couple, of, later, 0): {days: 34}

Using that we can see both sides of word.
#### Probability
When we see a word that is not in the list of existing words, we take its all n-gram contexts.

Example:
"a couple of dys later"
contexts: (of, later), (couple, of, later, 0)

At the the start all words have their own weight, which is initially equal to e.

After that, taking into account the found contexts, we increase the weight of those words that are affected by these contexts. To do this, we find the sum of all the words for a given context(sum), as well as the number of specific words whose weight we want to increase(num).

Next, we take the weight of the found word and divide it by (sum - num) / sum, pre-adding +1 to sum to avoid zero-division when there is only one type of word in the context and sum = num. It is also worth considering that 0 < (sum - num) / sum <= 1, so if we found context with given word, its weight will be increased depending on the frequency with which a given word appears in a given context. Thus, if there is no context, we do not do anything with the weight of the words, and if there is a deeper context, we continue to increase the weight, so words with a deeper context will have more weight. This allows us to process contexts of any depth without affecting words that do not have them.

After that, we find the Levenshtein distance from the unidentified word to the existing words, that will help us to find the lexographically closest words and rely less on context.

Finally, the total weight is calculated using the formula: log(weight of word) * exp(-Levenshtein distance).
log(weight of word) to get rid of oversaturated samples that will give out too much weight, also weigh of word >= e, so log(weight of word) >= 1.
exp(-Levenshtein distance) to increase the weight of words that are close lexographically, correcting several letters instead of selecting words based heavily on context
 

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity (or just take another dataset). Compare your solution to the Norvig's corrector, and report the accuracies.

In [107]:
import re
from collections import Counter


def words(text): return re.findall(r'\w+', text.lower())


WORDS = Counter(words(open('fivegrams.txt').read()))


def Norwig(text):
    def P(word, N=sum(WORDS.values())):
        "Probability of `word`."
        return WORDS[word] / N

    def correction(word):
        "Most probable spelling correction for word."
        return max(candidates(word), key=P)

    def candidates(word):
        "Generate possible spelling corrections for word."
        return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

    def known(words):
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in WORDS)

    def edits1(word):
        "All edits that are one edit away from `word`."
        letters = 'abcdefghijklmnopqrstuvwxyz'
        splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
        deletes = [L + R[1:] for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
        replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
        inserts = [L + c + R for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(word):
        "All edits that are two edits away from `word`."
        return (e2 for e1 in edits1(word) for e2 in edits1(e1))

    return ' '.join(correction(i) for i in text.split())

In [108]:
Norwig('tve forest wos alive with the saunds')

'the forest was alive with the sounds'

In [109]:
# Your code here
with open('test.txt') as f:
    sentences = [normalize(sent, True) for sent in f.read().split('.') if sent != '']

In [110]:
from random import random, randint


def ruined_sentence(text, p=0.3):
    text = text.split()
    for i in range(len(text)):
        if random() > p: continue
        for j in range(round(6 - log(randint(2, 32), 2))):
            rnd = random()
            text[i] = list(text[i])
            k = randint(0, len(text[i]) - 1)
            if rnd < 0.33 and len(text[i]) > 1:
                text[i].pop(k)
            elif rnd < 0.66:
                text[i].insert(k, chr(randint(ord('a'), ord('z'))))
            else:
                text[i][k] = chr(randint(ord('a'), ord('z')))
            text[i] = ''.join(text[i])
    return ' '.join(text)

In [111]:
ruined_sentence(sentences[0])

'te forest was alive with the sounds of nature ja symphony of chirkspng birds rustling leaves aund whpze occasional eknfa of a twig underfoot'

In [112]:
accuracy1 = accuracy2 = 0

for sent in sentences:
    print('Original')
    print(sent)
    original = sent.split()
    sent = ruined_sentence(sent)
    print('With mistakes')
    print(sent)
    res1 = correct_text(sent)
    acc1 = round(sum(i == j for i, j in zip(original, res1.split())) / len(original), 4)
    accuracy1 += acc1
    print('Our accuracy:', acc1)
    print(res1)
    res2 = Norwig(sent)
    acc2 = round(sum(i == j for i, j in zip(original, res2.split())) / len(original), 4)
    accuracy2 += acc2
    print('Norwig accuracy:', acc2)
    print(res2)
    print()

Original
the forest was alive with the sounds of nature a symphony of chirping birds rustling leaves and the occasional snap of a twig underfoot
With mistakes
hm forest was alive with be okcnds of nmalqri a symphony of chirping birds rustling leaves uanc the mocaasional srap of wt qti underfoot
Our accuracy: 0.5833
hm forest was alive with be kinds of nmai a symphony of chipping birds rustling leaves anc the occasional wrap of wt qui underwood
Norwig accuracy: 0.5833
hm forest was alive with be kinds of nmalqri a symphony of chipping birds rustling leaves anc the occasional wrap of wt qui underwood

Original
the air was thick with the earthy scent of moss and damp soil a testament to the recent rain that had swept through the area
With mistakes
the wair was fhicfk with the earthy scent of moss apqd damp soil vz testament to the recent gin iat ha swept atrouh fthe area
Our accuracy: 0.72
the air was thick with the earthly scent of moss tpwd damp soil vu testament to the recent gin wiat 

In [115]:
print('Our overall accuracy:', round(accuracy1 / len(sentences), 4))
print('Norwig overall accuracy:', round(accuracy2 / len(sentences), 4))

Our overall accuracy: 0.7695
Norwig overall accuracy: 0.7713


#### Useful resources (also included in the archive in moodle):

1. [Possible dataset with N-grams](https://www.ngrams.info/download_coca.asp)
2. [Damerau–Levenshtein distance](https://en.wikipedia.org/wiki/Damerau–Levenshtein_distance#:~:text=Informally%2C%20the%20Damerau–Levenshtein%20distance,one%20word%20into%20the%20other.)